In [1]:
using HumanoidLCMSim
using RigidBodySim
using RigidBodyDynamics
using ValkyrieRobot
using OrdinaryDiffEq
using DiffEqCallbacks
using RigidBodyTreeInspector
using RigidBodyDynamics.Contact

In [2]:
const DRAKE_DIR = "/home/twan/code/drake/"

"/home/twan/code/drake/"

In [3]:
val = Valkyrie()
mechanism = val.mechanism
remove_fixed_tree_joints!(mechanism);

In [4]:
# add environment
rootframe = root_frame(mechanism)
ground = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism, ground);

In [5]:
# Create visualizer
any_open_visualizer_windows() || (new_visualizer_window(); sleep(1));
vis = Visualizer()[:valkyrie]
setgeometry!(vis, mechanism, RigidBodyTreeInspector.parse_urdf(ValkyrieRobot.urdfpath(), mechanism;
    package_path = [ValkyrieRobot.packagepath()]))

In [6]:
feet = Dict(Sides.left => findbody(mechanism, "leftFoot"), Sides.right => findbody(mechanism, "rightFoot"))
hands = Dict(Sides.left => findbody(mechanism, "leftPalm"), Sides.right => findbody(mechanism, "rightPalm"))
robot_info = HumanoidRobotInfo(mechanism, feet, hands, parse_actuators(mechanism, ValkyrieRobot.urdfpath()));

In [7]:
state = MechanismState(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

In [8]:
function initialize(state::MechanismState, val::Valkyrie, vis::Visualizer)
    zero!(state)
    for side in instances(Side)
        set_configuration!(state, findjoint(mechanism, "$(side)KneePitch"), [1.205])
        set_configuration!(state, findjoint(mechanism, "$(side)HipPitch"), [-0.49])
        set_configuration!(state, findjoint(mechanism, "$(side)AnklePitch"), [-0.71])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderPitch"), [0.300196631343025])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderRoll"), [Sides.flipsign_if_right(-1.25, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ElbowPitch"), [Sides.flipsign_if_right(-0.785398163397448, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ForearmYaw"), [1.571])
    end
    set_configuration!(state, val.basejoint, [1; 0; 0; 0; 0; 0; 1.025])
    settransform!(vis, state)
    nothing
end

initialize (generic function with 1 method)

In [9]:
lcmcontroller = LCMController(robot_info)
controller = PeriodicController(0.003, lcmcontroller);

In [10]:
function drake_controller_proc(f)
    proc = spawn(Cmd(`bazel-bin/examples/humanoid_controller/valkyrie_balancing_demo`, dir = DRAKE_DIR))
    sleep(0.1)
    ret = try
        f()
    finally
        kill(proc)
    end
    ret
end

drake_controller_proc (generic function with 1 method)

In [11]:
sol = drake_controller_proc() do
    initialize(state, val, vis)
    final_time = 10.
    problem = ODEProblem(state, (0., final_time), controller)
    @time solve(problem, Tsit5(), abs_tol = 1e-8, dt = controller.Δt, callback = CallbackSet(vis, state, max_fps = 30.))
end;

 21.517458 seconds (8.59 M allocations: 502.557 MiB, 2.80% gc time)


In [12]:
# t = Ref(0.)
# drake_controller_proc() do
#     HumanoidLCMSim.publish_robot_state(lcmcontroller, t[], state)
#     sleep(0.1)
#     @time for i = 1 : 1000
#         t[] += controller.Δt
#         HumanoidLCMSim.publish_robot_state(lcmcontroller, t[], state)
#         LCMCore.handle(lcmcontroller.lcm, Dates.Second(1))
#     end
# end

In [13]:
# Profile.clear()
# drake_controller_proc() do
#     initialize(state, val, vis)
#     problem = ODEProblem(state, (0., 4.), controller)
#     @profile solve(problem, Tsit5(), abs_tol = 1e-8, dt = controller.Δt)
# end
# using ProfileView
# ProfileView.view()

In [14]:
# animate(vis, state, sol)